# Backtest

In [1]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "size" : 45, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2022", "end" : "2023"
          }


env = Env(**config)

params_1 = {"symbol" : "BTC", "allocation" : 100, "env" : env}
params_2 = {"symbol" : "TWT", "allocation" : 100, "env" : env}

agent_1 = Agent(**params_1)
#agent_2 = Agent(**params_2)

In [2]:
agent_1.env.market.db.engine

Engine(sqlite:///C:\Users\cc\Desktop\CedAlgo\database\database_1d.db)

In [3]:
agent_1.run_episode()
#agent_2.run_episode()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
agent_1.trades_data["BTC"]

{'all':                price  quantity  position   side  \
 date                                              
 2022-02-14  42535.94  0.000000         0   None   
 2022-02-15  44544.86  0.000000         0   None   
 2022-02-16  43873.56 -0.002279        -1  SHORT   
 2022-02-17  40515.70 -0.002279        -1  SHORT   
 2022-02-18  39974.44 -0.004559         0   None   
 ...              ...       ...       ...    ...   
 2023-10-09  27590.12 -0.002959        -1  SHORT   
 2023-10-10  27390.12 -0.002959        -1  SHORT   
 2023-10-11  26875.52 -0.002959        -1  SHORT   
 2023-10-12  26759.63 -0.002959        -1  SHORT   
 2023-10-13  26862.00 -0.005918         0   None   
 
                                     state    in_value  out_value       value  \
 date                                                                           
 2022-02-14      ('-', None, False, False)    0.000000   0.000000    0.000000   
 2022-02-15      ('-', None, False, False)    0.000000   0.000000    0.0

In [ ]:
agent_1.trades_data["BTC"]["all"].head()

,price,quantity,position,side,state,in_value,out_value,value,pnl,pnl_pct,symbol,status,value2,ret_price,price_cum,gp,rets,loss,recovery
date,,,,,,,,,,,,,,,,,,,
2022-02-14,42535.94,0.000000,0,None,"('-', None, False, False)",0.0,0.000000,0.000000,0.000000,0.000000,BTC,-,0.000000,NaN,NaN,0.000000,0.000000,0.0,0.0
2022-02-15,44544.86,0.000000,0,None,"('-', None, False, False)",0.0,0.000000,0.000000,0.000000,0.000000,BTC,-,0.000000,0.047229,1.047229,0.000000,0.000000,0.0,0.0
2022-02-16,43873.56,-0.002279,-1,SHORT,"('Open', 'SHORT', 'tp', 'sl')",100.0,0.000000,100.000000,0.000000,0.000000,BTC,Open,100.000000,-0.015070,1.031447,0.000000,inf,0.0,0.0
2022-02-17,40515.70,-0.002279,-1,SHORT,"('-', None, False, False)",100.0,0.000000,107.653493,7.653493,0.076535,BTC,-,107.653493,-0.076535,0.952505,7.653493,0.076535,0.0,0.0
2022-02-18,39974.44,-0.004559,0,None,"('Close', 'LONG', 'tp', 'sl')",0.0,91.112825,-182.225650,8.887175,0.088872,BTC,Close,-91.112825,-0.013359,0.939780,0.000000,0.000000,0.0,0.0


In [ ]:
agent_1.postindicator

[{'all': {'Total Trades': 0,
   'Total Pnl': 7.653493356819013,
   'Win Rate': 0,
   'Expentancy': nan,
   'Amoung Win': 7.653493356819013,
   'Amoung Loss': 0.0,
   'Avg Win': 7.653493356819013,
   'Avg Loss': nan,
   'Profit Factor': inf},
  'date': Timestamp('2022-02-18 00:00:00'),
  'symbol': 'BTC'},
 {'all': {'Total Trades': 0,
   'Total Pnl': 8.688868793304238,
   'Win Rate': 0,
   'Expentancy': -3.60319844173614,
   'Amoung Win': 30.308059443721078,
   'Amoung Loss': -21.61919065041684,
   'Avg Win': 5.05134324062018,
   'Avg Loss': -3.60319844173614,
   'Profit Factor': 1.4019053688828405},
  'date': Timestamp('2022-03-02 00:00:00'),
  'symbol': 'BTC'},
 {'all': {'Total Trades': 0,
   'Total Pnl': 8.770597347232439,
   'Win Rate': 0,
   'Expentancy': -3.6309177041420133,
   'Amoung Win': 41.44885668451056,
   'Amoung Loss': -32.67825933727812,
   'Avg Win': 4.605428520501174,
   'Avg Loss': -3.6309177041420133,
   'Profit Factor': 1.2683924274151064},
  'date': Timestamp('2022-

In [ ]:
env.journal.portfolio_data

,date,risk_value,available_value,capital
0,2022-02-14,0.000000,100.000000,100.000000
1,2022-02-15,0.000000,100.000000,100.000000
2,2022-02-16,100.000000,0.000000,100.000000
3,2022-02-17,107.653493,0.000000,107.653493
4,2022-02-18,-182.225650,91.112825,-91.112825
...,...,...,...,...
606,2023-10-13,-158.980104,79.490052,-79.490052
607,2023-10-14,79.433709,0.000000,79.433709
608,2023-10-15,80.326095,0.000000,80.326095
609,2023-10-16,84.309630,0.000000,84.309630
